<a href="https://colab.research.google.com/github/abhinivesh-s/Semi-Supervised-Text/blob/main/Semi_Supervised_sklearn_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle

In [ ]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.16.2
0.24.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.16.2
import numpy as np

     |████████████████████████████████| 17.3MB 318kB/s 
ERROR: xarray 0.18.0 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.16.2 which is incompatible.
ERROR: pyarrow 3.0.0 has requirement numpy>=1.16.6, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.2.1 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [ ]:
!pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

     |████████████████████████████████| 22.3MB 56.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
!ls

/content/drive/My Drive/Colab Notebooks/MixText-master
 code	     LICENSE			'Running MixText v5 ISIS Data.ipynb'
 data	     README.md			'Semi-Supervised sklearn.ipynb'
 data_temp  'Running MixText v3.ipynb'	 word2vec_model.bin


In [ ]:
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = 1000   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs


class loader_labeled(Dataset):
    # Data loader for labeled data
    def __init__(self, dataset_text, dataset_label, tokenizer, max_seq_len, aug=False):
        self.tokenizer = tokenizer
        self.text = dataset_text
        self.labels = dataset_label
        self.max_seq_len = max_seq_len

        self.aug = aug
        self.trans_dist = {}

        if aug:
            print('Aug train data by back translation of German')
            self.en2de = torch.hub.load(
                'pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
            self.de2en = torch.hub.load(
                'pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

    def __len__(self):
        return len(self.labels)

    def augment(self, text):
        if text not in self.trans_dist:
            self.trans_dist[text] = self.de2en.translate(self.en2de.translate(
                text,  sampling=True, temperature=0.9),  sampling=True, temperature=0.9)
        return self.trans_dist[text]

    def get_tokenized(self, text):
        tokens = self.tokenizer.tokenize(text)
        if len(tokens) > self.max_seq_len:
            tokens = tokens[:self.max_seq_len]
        length = len(tokens)

        encode_result = self.tokenizer.convert_tokens_to_ids(tokens)
        padding = [0] * (self.max_seq_len - len(encode_result))
        encode_result += padding

        return encode_result, length

    def __getitem__(self, idx):
        if self.aug:
            text = self.text[idx]
            text_aug = self.augment(text)
            text_result, text_length = self.get_tokenized(text)
            text_result2, text_length2 = self.get_tokenized(text_aug)
            return ((torch.tensor(text_result), torch.tensor(text_result2)), (self.labels[idx], self.labels[idx]), (text_length, text_length2))
        else:
            text = self.text[idx]
            tokens = self.tokenizer.tokenize(text)
            if len(tokens) > self.max_seq_len:
                tokens = tokens[:self.max_seq_len]
            length = len(tokens)
            encode_result = self.tokenizer.convert_tokens_to_ids(tokens)
            padding = [0] * (self.max_seq_len - len(encode_result))
            encode_result += padding
            return (torch.tensor(encode_result), self.labels[idx], length)


class loader_unlabeled(Dataset):
    # Data loader for unlabeled data
    def __init__(self, dataset_text, unlabeled_idxs, tokenizer, max_seq_len, aug=None):
        self.tokenizer = tokenizer
        self.text = dataset_text
        self.ids = unlabeled_idxs
        self.aug = aug
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.text)

    def get_tokenized(self, text):
        tokens = self.tokenizer.tokenize(text)
        if len(tokens) > self.max_seq_len:
            tokens = tokens[:self.max_seq_len]
        length = len(tokens)
        encode_result = self.tokenizer.convert_tokens_to_ids(tokens)
        padding = [0] * (self.max_seq_len - len(encode_result))
        encode_result += padding
        return encode_result, length

    def __getitem__(self, idx):
        if self.aug is not None:
            u, v, ori = self.aug(self.text[idx], self.ids[idx])
            encode_result_u, length_u = self.get_tokenized(u)
            encode_result_v, length_v = self.get_tokenized(v)
            encode_result_ori, length_ori = self.get_tokenized(ori)
            return ((torch.tensor(encode_result_u), torch.tensor(encode_result_v), torch.tensor(encode_result_ori)), (length_u, length_v, length_ori))
        else:
            text = self.text[idx]
            encode_result, length = self.get_tokenized(text)
            return (torch.tensor(encode_result), length)


In [ ]:
%%time
import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data


CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.8 µs


In [ ]:
def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    lemmatizer = WordNetLemmatizer()
#     train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

In [ ]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index2word)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(reviews, model, num_features):
    """
    Calculate average feature vectors for all reviews
    """
    counter = 0
    review_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')  # pre-initialize (for speed)
    
    for review in reviews:
#         print(counter)
        review_feature_vecs[counter] = make_feature_vec(review, model, num_features)
        counter = counter + 1
    return review_feature_vecs


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
from gensim.models import KeyedVectors


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def get_data_tfidf(n, arg1=1):

  n_labeled_per_class = n       #10, 200, 500, 1000, 2400
  unlabeled_per_class = 1000


  data_path = './data/imdb_data/'
  train_df = pd.read_csv(data_path+'train.csv', header=None)
  test_df = pd.read_csv(data_path+'test.csv', header=None)
  print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class*2)
  print(train_df.head(5))


  train_labels = np.array([v for v in train_df[1]])
  train_text = np.array([v for v in train_df[0]])
  print(train_labels[:3])
  print(train_text[:3])

  test_labels = np.array([u for u in test_df[1]])
  test_text = np.array([v for v in test_df[0]])




  n_labels = 2
  # Split the labeled training set, unlabeled training set, development set
  train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
      train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

  print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))
  


  df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
  print(df_train.shape)
  # df_train.head()

  df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
  print(df_test.shape)

  if arg1!=1:
    return df_test
  # df_test.head()

  df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
  print(df_unlabeled.shape)
  # df_unlabeled.head()

  clean_train_df = clean(df_train)
  clean_test_df = clean(df_test)
  clean_unlabeled_df = clean(df_unlabeled)

  texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))

  # Create Corpus
  # model = word2vec.Word2Vec(texts, min_count=1)

  NGRAM_RANGE = (1, 2)
  TOP_K = 20000
  TOKEN_MODE = 'word'
  MIN_DOC_FREQ = 2

  kwargs = {
      'ngram_range' : NGRAM_RANGE,
      'dtype' : 'int32',
      'strip_accents' : 'unicode',
      'decode_error' : 'replace',
      'analyzer' : TOKEN_MODE,
      'min_df' : MIN_DOC_FREQ,
  }

  tfidf_vectorizer = TfidfVectorizer(**kwargs)


  ##save the modeled words produced from Word2Vec
  # model.save('word2vec_model.bin')


  # loaded_model=KeyedVectors.load('word2vec_model.bin')
  # print("Loaded word2vec model:", loaded_model)

  # num_features = 100 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
  # feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
  # feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
  # feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)

  feature_vec_full_train = tfidf_vectorizer.fit_transform(texts)

  feature_vec_train = tfidf_vectorizer.transform(clean_train_df['review'])
  feature_vec_test = tfidf_vectorizer.transform(clean_test_df['review'])
  feature_vec_unlabeled = tfidf_vectorizer.transform(clean_unlabeled_df['review'])


  TOP_K = 20000
  selector = SelectKBest(f_classif, k=min(TOP_K, feature_vec_train.shape[1]))

  selector.fit(feature_vec_train, np.array([i for i in list(df_train.sentiment)]))

  feature_vec_train = selector.transform(feature_vec_train).astype('float32').toarray()
  feature_vec_test = selector.transform(feature_vec_test).astype('float32').toarray()
  feature_vec_unlabeled = selector.transform(feature_vec_unlabeled).astype('float32').toarray()


  print("TYPE", type(feature_vec_train))
  feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
  print("Shape of train data: ",feature_vec_full_train.shape)

  labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

  return feature_vec_full_train, feature_vec_test, labels, test_labels, df_test

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!ls

 code	     LICENSE			'Running MixText v5 ISIS Data.ipynb'
 data	     README.md			'Semi-Supervised sklearn.ipynb'
 data_temp  'Running MixText v3.ipynb'	 word2vec_model.bin


In [ ]:
# For tf-idf

n_list = [10, 200, 500, 1000, 2400]
# n_list=[10]

for i in n_list:

  train_vec, test_vec, labels, test_labels, df_test = get_data_tfidf(i)

  print("TRAIN VEC: ", train_vec.shape)
  np.save('data_temp/tf-idf/train_vec_'+str(i), train_vec)
  np.save('data_temp/tf-idf/test_vec_'+str(i), test_vec)
  np.save('data_temp/tf-idf/labels_'+str(i), labels)
  np.save('data_temp/tf-idf/test_labels_'+str(i), test_labels)

  print(i, 'SAVED!!')

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (2020, 20000)
TRAIN VEC:  (2020, 20000)
10 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  400
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin becaus

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (2400, 20000)
TRAIN VEC:  (2400, 20000)
200 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  1000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin beca

...

TYPE <class 'numpy.ndarray'>
Shape of train data:  (3000, 20000)
TRAIN VEC:  (3000, 20000)
500 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  2000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin beca

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (4000, 20000)
TRAIN VEC:  (4000, 20000)
1000 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  4800
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin bec

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (6800, 20000)
TRAIN VEC:  (6800, 20000)
2400 SAVED!!


In [ ]:
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  print("HELLO!", train_vec.shape)

  label_prop_model = LabelSpreading()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
train_vec = np.load('data_temp/tf-idf/train_vec_'+str(10)+'.npy')
train_vec

array([[0.04196495, 0.07739466, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02940054, 0.        , 0.05313623, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 54.040
Accuracy for  400  labeled data is: 67.456
Accuracy for  1000  labeled data is: 68.272
Accuracy for  2000  labeled data is: 52.896
Accuracy for  4800  labeled data is: 51.088


In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelPropagation()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  20  labeled data is: 50.824
Read data successful (37500, 2) (12500, 2) n_labeled =  400
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not del

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  400  labeled data is: 49.272
Read data successful (37500, 2) (12500, 2) n_labeled =  1000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not d

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  1000  labeled data is: 49.592
Read data successful (37500, 2) (12500, 2) n_labeled =  2000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not 

In [ ]:
print('---------Label Propagation---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Propagation---------
Accuracy for  20  labeled data is: 50.824
Accuracy for  400  labeled data is: 49.272
Accuracy for  1000  labeled data is: 49.592
Accuracy for  2000  labeled data is: 78.264
Accuracy for  4800  labeled data is: 52.776


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 


  label_prop_model = SelfTrainingClassifier(svc)
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
print('---------Self Training Classifier with SVM---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Self Training Classifier with SVM---------
Accuracy for  20  labeled data is: 63.672
Accuracy for  400  labeled data is: 76.496
Accuracy for  1000  labeled data is: 76.712
Accuracy for  2000  labeled data is: 80.968
Accuracy for  4800  labeled data is: 79.168


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Without Unlabeled Data

n_list = [10, 200, 500, 1000, 2400]
n_list = [10, 200, 500, 1000]
acc_list_2 = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 

  print("SHAPE: ", train_vec.shape, "SIZE: ", labels.shape)
  print("New SHAPE: ", train_vec[:2*i].shape, "New SIZE: ", labels[:2*i].shape, np.unique(labels), np.unique(labels[:2*i]))

  svc.fit(train_vec[:2*i], labels[:2*i])

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = svc.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list_2[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list_2[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 63.672
Accuracy for  400  labeled data is: 76.496
Accuracy for  1000  labeled data is: 76.712
Accuracy for  2000  labeled data is: 80.968


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
from gensim.models import KeyedVectors

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def get_data(n, arg1=1):

  n_labeled_per_class = n       #10, 200, 500, 1000, 2400
  unlabeled_per_class = 1000


  data_path = './data/imdb_data/'
  train_df = pd.read_csv(data_path+'train.csv', header=None)
  test_df = pd.read_csv(data_path+'test.csv', header=None)
  print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class*2)
  print(train_df.head(5))


  train_labels = np.array([v for v in train_df[1]])
  train_text = np.array([v for v in train_df[0]])
  print(train_labels[:3])
  print(train_text[:3])

  test_labels = np.array([u for u in test_df[1]])
  test_text = np.array([v for v in test_df[0]])




  n_labels = 2
  # Split the labeled training set, unlabeled training set, development set
  train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
      train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

  print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))
  


  df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
  print(df_train.shape)
  # df_train.head()

  df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
  print(df_test.shape)

  if arg1!=1:
    return df_test
  # df_test.head()

  df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
  print(df_unlabeled.shape)
  # df_unlabeled.head()

  clean_train_df = clean(df_train)
  clean_test_df = clean(df_test)
  clean_unlabeled_df = clean(df_unlabeled)

  texts = (clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)).apply(lambda x:x.split())

  # Create Corpus
  embedding_vector_size = 300
  model = word2vec.Word2Vec(sentences = texts, size = embedding_vector_size, min_count=3, window=5, workers=4)

  ##save the modeled words produced from Word2Vec
  model.save('word2vec_model.bin')


  loaded_model=KeyedVectors.load('word2vec_model.bin')
  print("Loaded word2vec model:", loaded_model)

  num_features = 300 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
  feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
  feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
  feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)



  feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
  print("Shape of train data: ",feature_vec_full_train.shape)

  labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

  return feature_vec_full_train, feature_vec_test, labels, test_labels, df_test


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# For word2vec

n_list = [10, 200, 500, 1000, 2400]

for i in n_list:

  train_vec, test_vec, labels, test_labels, df_test = get_data(i)

  np.save('data_temp/train_vec_'+str(i), train_vec)
  np.save('data_temp/test_vec_'+str(i), test_vec)
  np.save('data_temp/labels_'+str(i), labels)
  np.save('data_temp/test_labels_'+str(i), test_labels)

  print(i, 'SAVED!!')

SyntaxError: ignored

In [ ]:
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelSpreading()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 49.584
Accuracy for  400  labeled data is: 53.720
Accuracy for  1000  labeled data is: 54.448
Accuracy for  2000  labeled data is: 54.480
Accuracy for  4800  labeled data is: 54.536


In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelPropagation()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Label Propagation---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Propagation---------
Accuracy for  20  labeled data is: 49.256
Accuracy for  400  labeled data is: 51.400
Accuracy for  1000  labeled data is: 54.152
Accuracy for  2000  labeled data is: 54.272
Accuracy for  4800  labeled data is: 54.400


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 


  label_prop_model = SelfTrainingClassifier(svc)
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Self Training Classifier with SVM---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Self Training Classifier with SVM---------
Accuracy for  20  labeled data is: 52.928
Accuracy for  400  labeled data is: 53.400
Accuracy for  1000  labeled data is: 54.120
Accuracy for  2000  labeled data is: 54.944
Accuracy for  4800  labeled data is: 55.224


In [ ]:
import sklearn
sklearn.show_versions()


System:
    python: 3.7.10 (default, May  3 2021, 02:48:31)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 19.3.1
   setuptools: 56.1.0
      sklearn: 0.24.2
        numpy: 1.19.5
        scipy: 1.4.1
       Cython: 0.29.23
       pandas: 1.1.5
   matplotlib: 3.2.2
       joblib: 1.0.1
threadpoolctl: 2.1.0

Built with OpenMP: True


In [ ]:
n_labeled_per_class = 200       #10, 200, 500, 1000, 2400
unlabeled_per_class = 1000

In [ ]:
data_path = './data/imdb_data/'
train_df = pd.read_csv(data_path+'train.csv', header=None)
test_df = pd.read_csv(data_path+'test.csv', header=None)
print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class)
print(train_df.head(5))

Read data successful (37500, 2) (12500, 2) n_labeled =  200
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1


In [ ]:
train_labels = np.array([v for v in train_df[1]])
train_text = np.array([v for v in train_df[0]])
print(train_labels[:3])
print(train_text[:3])

test_labels = np.array([u for u in test_df[1]])
test_text = np.array([v for v in test_df[0]])

[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to keep my interest. <br /><br />To be honest I did continue watching...Watching with hopes to see if anything suddenly clicked. It didn\'t. So, stylish as it is, I wouldn\'t recommend this movie. BTW, it seems odd to see Mary Tyler Moore as the principal. She\'s truly miscast, I hope the paycheck was inordinately big.'
 "This is the kind of movie that wants to be good but sucks. First thing, wha

In [ ]:
n_labels = 2
# Split the labeled training set, unlabeled training set, development set
train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
    train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

# Build the dataset class for each set
# train_labeled_dataset = loader_labeled(
#     train_text[train_labeled_idxs], train_labels[train_labeled_idxs], tokenizer, max_seq_len, train_aug)
# train_unlabeled_dataset = loader_unlabeled(
#     train_text[train_unlabeled_idxs], train_unlabeled_idxs, tokenizer, max_seq_len, Translator(data_path))
# val_dataset = loader_labeled(
#     train_text[val_idxs], train_labels[val_idxs], tokenizer, max_seq_len)
# test_dataset = loader_labeled(
#     test_text, test_labels, tokenizer, max_seq_len)

print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
    train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

#Labeled: 400, Unlabeled 2000, Val 4000, Test 12500


In [ ]:
df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
print(df_train.shape)
df_train.head()

(400, 2)


,review,sentiment
0,"okay, let's cut to the chase - there's no way ...",0
1,I saw Hurlyburly on Broadway and liked it a gr...,0
2,"First of all, I'd like to tell you that I'm in...",0
3,Harlan Knowles (Lance Henriksen) brings a grou...,0
4,Ugh. Pretty awful.<br /><br />Linnea Quigley g...,0


In [ ]:
df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
print(df_test.shape)
df_test.head()

(12500, 2)


,review,sentiment
0,I really liked this Summerslam due to the look...,1
1,Not many television shows appeal to quite as m...,1
2,The film quickly gets to a major chase scene w...,0
3,Jane Austen would definitely approve of this o...,1
4,Expectations were somewhat high for me when I ...,0


In [ ]:
df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
print(df_unlabeled.shape)
df_unlabeled.head()

(2000, 2)


,review,sentiment
0,"This is the epitome of bad 80's film-making, u...",0
1,(Only minor spoilers except as noted).<br /><b...,0
2,'Dead Letter Office' is a low-budget film abou...,0
3,"Anyone who visited drive-ins in the 1950s, 60s...",0
4,Feature of early 21 century cinema of lets pit...,0


In [ ]:
#Lemmatize the dataset
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
%%time
clean_train_df = clean(df_train)
clean_test_df = clean(df_test)
clean_unlabeled_df = clean(df_unlabeled)

CPU times: user 13.2 s, sys: 221 ms, total: 13.4 s
Wall time: 13.3 s


In [ ]:
# texts = (df_train['review'].append(df_test['review'], ignore_index=True)).apply(lambda x:x.split())
texts = (clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)).apply(lambda x:x.split())

In [ ]:
%%time
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
# Create Corpus
model = word2vec.Word2Vec(texts, min_count=1)

CPU times: user 15.6 s, sys: 199 ms, total: 15.8 s
Wall time: 10.8 s


In [ ]:
word_li=list(model.wv.vocab)
print(word_li[:5])

['okay', 'lets', 'cut', 'to', 'the']


In [ ]:
##save the modeled words produced from Word2Vec
model.save('word2vec_model.bin')

In [ ]:
from gensim.models import KeyedVectors
loaded_model=KeyedVectors.load('word2vec_model.bin')
print(loaded_model)

Word2Vec(vocab=39344, size=100, alpha=0.025)


In [ ]:
%%time
num_features = 100 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


CPU times: user 2min 7s, sys: 1.51 s, total: 2min 9s
Wall time: 2min 8s


In [ ]:
print(feature_vec_train.shape, feature_vec_test.shape, feature_vec_unlabeled.shape)

(400, 100) (12500, 100) (2000, 100)


In [ ]:
print(type(feature_vec_train), type(feature_vec_test), type(feature_vec_unlabeled))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
feature_vec_full_train.shape

(2400, 100)

In [ ]:
labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])
print(labels)
len(labels)

[ 0  0  0 ... -1 -1 -1]


2400

In [ ]:
#n_labeled=20
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

Accuracy: 49.592


In [ ]:
#n_labeled=400
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

Accuracy: 53.744
CPU times: user 3.31 s, sys: 1.82 s, total: 5.14 s
Wall time: 2.74 s


In [ ]:
#n_labeled=1000
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

In [ ]:
#n_labeled=2000
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

In [ ]:
#n_labeled=4800
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))